In [67]:
import random
class Sentiment:
    NEGATIVE='NEGATIVE'
    POSITIVE='POSITIVE'
    NEUTRAL='NEUTRAL'
    
class Review:
    def __init__(self,text,score):
        self.text=text
        self.score=score
        self.sentiment=self.get_sentiment()
        
        
    def get_sentiment(self):
        if self.score<=2:
            return Sentiment.NEGATIVE
        elif self.score==3:
            return Sentiment.NEUTRAL
        else:
            return Sentiment.POSITIVE

            
class ReviewContainer:
    def __init__(self,reviews):
        self.reviews=reviews
        
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
    
    def evenly_distribute(self):
        negative=list(filter(lambda x: x.sentiment==Sentiment.NEGATIVE,self.reviews))
        positve=list(filter(lambda x: x.sentiment==Sentiment.POSITIVE,self.reviews))
        positive_shrunk=positve[:len(negative)]
        self.reviews=negative+positive_shrunk
        random.shuffle(self.reviews)
        
        
        

In [22]:
import json

file ='Books_small_10000.json'
reviews=[]
with open(file) as f:
    for line in f:
        review =json.loads(line)
        reviews.append(Review(review['reviewText'],review['overall']))
        
reviews[7].text

"This is the First book in the Trilogy, and I'm looking forward to reading the second book.  I liked how the main characters interacted with famous characters in western history."

In [68]:
from sklearn.model_selection import train_test_split

training,test= train_test_split(reviews,test_size=0.33,random_state=42)

train_container=ReviewContainer(training)

test_container=ReviewContainer(test)


In [69]:
train_container.evenly_distribute()
train_x=train_container.get_text()
train_y=train_container.get_sentiment()

test_container.evenly_distribute()
test_x=test_container.get_text()
test_y=test_container.get_sentiment()

print(train_y.count(Sentiment.POSITIVE))
print(train_y.count(Sentiment.NEGATIVE))
print(len(train_y))


436
436
872


In [70]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
# vectorizer=CountVectorizer()
vectorizer=TfidfVectorizer()

train_x_vectors=vectorizer.fit_transform(train_x)
test_x_vectors=vectorizer.transform(test_x)

print(train_x_vectors.shape)




(872, 8906)


### svm


In [71]:
from sklearn import svm
clf_svm=svm.SVC(kernel='linear',C=1.5)
clf_svm.fit(train_x_vectors,train_y)
print(test_x[0])
clf_svm.predict(test_x_vectors[0])

Loved it, Jonas Saul is the best writer ever! Never a dull moment, fast paced till the end and leaves you wanting more.


array(['POSITIVE'], dtype='<U8')

### Decision Tree

In [75]:
from sklearn.tree import DecisionTreeClassifier

clf_dec=DecisionTreeClassifier()
clf_dec.fit(train_x_vectors,train_y)

clf_dec.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

### Logistic regression

In [77]:
from sklearn.linear_model import LogisticRegression

clf_log=LogisticRegression()
clf_log.fit(train_x_vectors,train_y)
clf_log.predict(test_x_vectors[0])


array(['POSITIVE'], dtype='<U8')

In [72]:
# mean accuracy
print(clf_svm.score(test_x_vectors,test_y))

0.8076923076923077


In [73]:
#F1 scores
from sklearn.metrics import f1_score
f1_score(test_y,clf_svm.predict(test_x_vectors), average=None,labels=[Sentiment.POSITIVE,Sentiment.NEUTRAL,Sentiment.NEGATIVE])


C:\Program Files\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Program Files\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


array([0.80769231, 0.        , 0.80769231])

In [74]:
from sklearn.model_selection import GridSearchCV
parameters={'kernel':('linear','rbf'),'C':(0.6,1,1.3,1.6,4,7,9)}
svc=svm.SVC()
clf=GridSearchCV(svc,parameters,cv=5)
clf.fit(train_x_vectors,train_y)

C:\Program Files\Anaconda\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Program Files\Anaconda\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Program Files\Anaconda\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Program Files\Anaconda\lib\site-packages\sklearn\svm\base.py:193: FutureW

C:\Program Files\Anaconda\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Program Files\Anaconda\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Program Files\Anaconda\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Program Files\Anaconda\lib\site-packages\sklearn\svm\base.py:193: FutureW

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': (0.6, 1, 1.3, 1.6, 4, 7, 9),
                         'kernel': ('linear', 'rbf')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [82]:
parameters={'C':(0.6,1,1.3,1.6)}
log=LogisticRegression()
clf_lrc=GridSearchCV(log,parameters,cv=5)
clf_lrc.fit(train_x_vectors,train_y)

C:\Program Files\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None, param_grid={'C': (0.6, 1, 1.3, 1.6)},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)